In [1]:
import org.apache.spark.SparkContext
import org.apache.spark.SparkConf
import org.apache.spark.rdd.RDD
import com.databricks.spark.csv
import org.apache.spark.mllib.feature.{StandardScaler, Normalizer}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.functions._
import org.apache.spark.sql.hive.HiveContext
import breeze.linalg.DenseVector
import breeze.linalg.DenseMatrix
import scala.collection._
import play.api.libs.json._


In [2]:
var credentials_1 = scala.collection.mutable.HashMap[String, String](
  "auth_url"->"https://identity.open.softlayer.com",
  "project"->"object_storage_7115a8c7_5ba1_47b4_84ad_60d394b80e0d",
  "project_id"->"d9baeb325dc54f4295d956b82f06a47f",
  "region"->"dallas",
  "user_id"->"4b2349419a524e65b0aa0c5ff155771c",
  "domain_id"->"494f75a1a8aa4655b93ac9e88be244a4",
  "domain_name"->"1055881",
  "username"->"admin_5880b7404a7aa61ca5826c7c8c569f27f8d377cf",
  "password"->"""hH)3(t8m1kyOdAU*""",
  "filename"->"testData4.csv",
  "container"->"notebooks",
  "tenantId"->"s1b2-40518d39c0c1ed-0c1e95d7674b"
)

In [2]:
var credentials_2 = scala.collection.mutable.HashMap[String, String](
  "auth_url"->"https://identity.open.softlayer.com",
  "project"->"object_storage_7115a8c7_5ba1_47b4_84ad_60d394b80e0d",
  "project_id"->"d9baeb325dc54f4295d956b82f06a47f",
  "region"->"dallas",
  "user_id"->"4b2349419a524e65b0aa0c5ff155771c",
  "domain_id"->"494f75a1a8aa4655b93ac9e88be244a4",
  "domain_name"->"1055881",
  "username"->"admin_5880b7404a7aa61ca5826c7c8c569f27f8d377cf",
  "password"->"""hH)3(t8m1kyOdAU*""",
  "filename"->"testData3.csv",
  "container"->"notebooks",
  "tenantId"->"s1b2-40518d39c0c1ed-0c1e95d7674b"
)

In [3]:
credentials_1("name")="keystone"

In [4]:
def setHadoopConfig(name: String, tenant: String, url: String, username: String, password: String, region: String) = {
    sc.hadoopConfiguration.set(f"fs.swift.service.$name.auth.url",url+"/v3/auth/tokens")
    sc.hadoopConfiguration.set(f"fs.swift.service.$name.auth.endpoint.prefix","endpoints")
    sc.hadoopConfiguration.set(f"fs.swift.service.$name.tenant",tenant)
    sc.hadoopConfiguration.set(f"fs.swift.service.$name.username",username)
    sc.hadoopConfiguration.set(f"fs.swift.service.$name.password",password)
    sc.hadoopConfiguration.setInt(f"fs.swift.service.$name.http.port",8080)
    sc.hadoopConfiguration.set(f"fs.swift.service.$name.region",region)
    sc.hadoopConfiguration.setBoolean(f"fs.swift.service.$name.public",true)
}


In [5]:
setHadoopConfig(credentials_1("name"), credentials_1("project_id"), credentials_1("auth_url"), credentials_1("user_id"), credentials_1("password"), credentials_1("region"))

In [6]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
//val hqlContext = new HiveContext(sc)
//val sqlContext = new SQLContext(sc)
//val hqlContext = new org.apache.spark.sql.hive.HiveContext(sc) 
import sqlContext.implicits._
import sqlContext._
import org.apache.spark.sql._

In [7]:
case class Task(issueName: String, source: String, userName: String, timeInDays: Integer, complexities: Integer, dayOfWeek: Integer)

In [8]:
var data = sc.textFile("swift://notebooks.keystone/testData4.csv")
data.count()

In [9]:
val allSplit = data.map(line => line.split(","))

In [10]:
val allData = allSplit.map(p => Task(p(0), p(1), p(2), p(3).toInt, p(4).toInt, p(5).toInt))

In [11]:
val allDF = allData.toDF()

In [12]:
val rowsRDD = allDF.rdd.map(r => (r.getString(0), r.getString(1), r.getString(2), r.getInt(3), r.getInt(4), r.getInt(5)))

rowsRDD.cache()

MapPartitionsRDD[7] at map at <console>:61

In [13]:
val vectors = allDF.rdd.map(r => Vectors.dense( r.getInt(3), r.getInt(4)))

In [14]:
vectors.cache()

MapPartitionsRDD[8] at map at <console>:61

In [15]:
allDF.registerTempTable("sfpd")

In [16]:
val userData = sqlContext.sql("SELECT userName, COUNT(userName) AS frequency FROM sfpd group by userName").collect()
val noOfClusters = userData.length
println(noOfClusters)

3


In [17]:
val kMeansModel = KMeans.train(vectors, noOfClusters, 15000)

In [18]:
val result = kMeansModel.clusterCenters.foreach(println)

[2.875,1.0]
[6.0,1.0]
[8.0,1.0]


In [19]:
val siic1 = kMeansModel.computeCost(vectors)
val indices = kMeansModel.predict(vectors)

In [20]:
val predictions = rowsRDD.map{r => (r._3, kMeansModel.predict(Vectors.dense(r._4, r._5) ))}

In [21]:
val predDF = predictions.toDF("userName", "CLUSTER")

In [22]:
println(predDF)

[userName: string, CLUSTER: int]


In [23]:
val t = allDF.join(predDF, "userName")

In [24]:
t.filter("CLUSTER = 0").show()

+--------+---------+-------+----------+------------+---------+-------+
|userName|issueName| source|timeInDays|complexities|dayOfWeek|CLUSTER|
+--------+---------+-------+----------+------------+---------+-------+
|   User1|    Task1|source1|         2|           1|        1|      0|
|   User1|    Task1|source1|         2|           1|        1|      0|
|   User1|    Task1|source1|         2|           1|        1|      0|
|   User1|    Task1|source1|         2|           1|        1|      0|
|   User1|    Task1|source1|         2|           1|        1|      0|
|   User1|    Task1|source1|         2|           1|        1|      0|
|   User1|    Task1|source1|         2|           1|        1|      0|
|   User1|    Task1|source1|         2|           1|        1|      0|
|   User1|    Task1|source1|         2|           1|        1|      0|
|   User1|    Task1|source1|         2|           1|        1|      0|
|   User1|    Task2|source1|         2|           1|        2|      0|
|   Us

In [ ]:
t.filter("CLUSTER = 1").show()

In [ ]:
t.filter("CLUSTER = 0").show()
t.filter("CLUSTER = 1").show()
t.filter("CLUSTER = 2").show()

In [ ]:
t.filter("CLUSTER = 0").describe().show()

In [25]:
val vectors1 = allDF.rdd.map(r => Vectors.dense( r.getInt(4), r.getInt(5)))

In [26]:
vectors1.cache()

MapPartitionsRDD[61] at map at <console>:61

In [27]:
val DataOfDays = sqlContext.sql("SELECT DISTINCT dayOfWeek FROM sfpd group by dayOfWeek").collect()
val noOfClusters1 = DataOfDays.length
println(noOfClusters1)

2


In [28]:
val kMeansModel1 = KMeans.train(vectors1, noOfClusters1, 15000)

In [29]:
val result = kMeansModel.clusterCenters.foreach(println)

[2.875,1.0]
[6.0,1.0]
[8.0,1.0]


In [50]:
val vectors1 = allDF.rdd.map(r => Vectors.dense( r.getInt(4), r.getInt(5)))
vectors1.cache()
val DataOfDays = sqlContext.sql("SELECT DISTINCT dayOfWeek FROM sfpd group by dayOfWeek").collect()
val noOfClusters1 = DataOfDays.length
println(noOfClusters1)
val kMeansModel1 = KMeans.train(vectors1, noOfClusters1, 15000)
val result = kMeansModel1.clusterCenters.foreach(println)

2
[1.0,1.0]
[1.0,2.0]


In [55]:
val predictions1 = rowsRDD.map{r => (r._1, kMeansModel1.predict(Vectors.dense(r._5, r._6) ))}
println(predictions1)
val predDF1 = predictions1.toDF("issueName", "CLUSTER")
val t1 = allDF.join(predDF1, "issueName")


MapPartitionsRDD[530] at map at <console>:71


In [58]:
t1.filter("CLUSTER = 0").collect().foreach(println)
t1.filter("CLUSTER = 1").collect().foreach(println)

[Task1,source1,User1,2,1,1,0]
[Task1,source1,User1,2,1,1,0]
[Task1,source1,User1,2,1,1,0]
[Task1,source1,User1,2,1,1,0]
[Task1,source1,User1,2,1,1,0]
[Task1,source1,User1,2,1,1,0]
[Task1,source1,User1,2,1,1,0]
[Task1,source1,User1,2,1,1,0]
[Task1,source1,User1,2,1,1,0]
[Task1,source1,User1,2,1,1,0]
[Task1,source1,User2,6,1,1,0]
[Task1,source1,User2,6,1,1,0]
[Task1,source1,User2,6,1,1,0]
[Task1,source1,User2,6,1,1,0]
[Task1,source1,User2,6,1,1,0]
[Task1,source1,User2,6,1,1,0]
[Task1,source1,User2,6,1,1,0]
[Task1,source1,User2,6,1,1,0]
[Task1,source1,User2,6,1,1,0]
[Task1,source1,User2,6,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,source1,User3,8,1,1,0]
[Task1,sou

In [2]:
%AddJar http://central.maven.org/maven2/org/plotly-scala/plotly-core_2.11/0.1.0/plotly-core_2.11-0.1.0.jar

Starting download from http://central.maven.org/maven2/org/plotly-scala/plotly-core_2.11/0.1.0/plotly-core_2.11-0.1.0.jar
Finished download of plotly-core_2.11-0.1.0.jar


In [12]:
%AddJar http://central.maven.org/maven2/co/theasi/plotly_2.10/0.1/plotly_2.10-0.1.jar

Using cached version of plotly_2.10-0.1.jar


In [13]:
%AddJar http://central.maven.org/maven2/co/theasi/plotly_2.11/0.1/plotly_2.11-0.1-sources.jar

Starting download from http://central.maven.org/maven2/co/theasi/plotly_2.11/0.1/plotly_2.11-0.1-sources.jar
Finished download of plotly_2.11-0.1-sources.jar


In [16]:
%AddJar http://central.maven.org/maven2/co/theasi/plotly_2.11/0.1/plotly_2.11-0.1-javadoc.jar

Starting download from http://central.maven.org/maven2/co/theasi/plotly_2.11/0.1/plotly_2.11-0.1-javadoc.jar
Finished download of plotly_2.11-0.1-javadoc.jar


In [24]:
%AddJar http://central.maven.org/maven2/org/plotly-scala/plotly-jupyter-scala_2.11/0.1.0/plotly-jupyter-scala_2.11-0.1.0-javadoc.jar

Starting download from http://central.maven.org/maven2/org/plotly-scala/plotly-jupyter-scala_2.11/0.1.0/plotly-jupyter-scala_2.11-0.1.0-javadoc.jar
Finished download of plotly-jupyter-scala_2.11-0.1.0-javadoc.jar


In [27]:
%AddJar http://central.maven.org/maven2/org/plotly-scala/plotly-jupyter-scala_2.11/0.1.0/plotly-jupyter-scala_2.11-0.1.0-sources.jar

Starting download from http://central.maven.org/maven2/org/plotly-scala/plotly-jupyter-scala_2.11/0.1.0/plotly-jupyter-scala_2.11-0.1.0-sources.jar
Finished download of plotly-jupyter-scala_2.11-0.1.0-sources.jar


In [28]:
%AddJar http://central.maven.org/maven2/org/plotly-scala/plotly-jupyter-scala_2.11/0.1.0/plotly-jupyter-scala_2.11-0.1.0.jar

Starting download from http://central.maven.org/maven2/org/plotly-scala/plotly-jupyter-scala_2.11/0.1.0/plotly-jupyter-scala_2.11-0.1.0.jar
Finished download of plotly-jupyter-scala_2.11-0.1.0.jar


In [2]:
%AddJar http://central.maven.org/maven2/co/theasi/plotly_2.10/0.1/plotly_2.10-0.1.jar

Using cached version of plotly_2.10-0.1.jar


In [3]:
import co.theasi.plotly

val gdpAmerica = Vector(12779.379640000001, 3822.1370840000004, 9065.800825, 36319.235010000004,
  13171.63885, 7006.580419, 9645.06142, 8948.102923, 6025.374752000001,
  6873.262326000001, 5728.353514, 5186.050003, 1201.637154,
  3548.3308460000003, 7320.880262000001, 11977.57496, 2749.320965,
  9809.185636, 4172.838464, 7408.905561, 19328.70901, 18008.50924,
  42951.65309, 10611.46299, 11415.805690000001)

val lifeExpectancyAmerica = Vector(75.32, 65.554, 72.39, 80.653, 78.553, 72.889,
  78.782, 78.273, 72.235, 74.994, 71.878, 70.259, 60.916, 70.198, 72.567,
  76.195, 72.899, 75.537, 71.752, 71.421, 78.746, 69.819, 78.242, 76.384, 73.747)

val labelAmerica = Vector(
  "Argentina",
  "Bolivia",
  "Brazil",
  "Canada",
  "Chile",
  "Colombia",
  "Costa Rica",
  "Cuba",
  "Dominican Republic",
  "Ecuador",
  "El Salvador",
  "Guatemala",
  "Haiti",
  "Honduras",
  "Jamaica",
  "Mexico",
  "Nicaragua",
  "Panama",
  "Paraguay",
  "Peru",
  "Puerto Rico",
  "Trinidad and Tobago",
  "United States",
  "Uruguay",
  "Venezuela"
)

val gdpEurope = Vector(5937.029525999999, 36126.4927, 33692.60508, 7446.298803, 10680.79282,
  14619.222719999998, 22833.30851, 35278.41874, 33207.0844, 30470.0167,
  32170.37442, 27538.41188, 18008.94444, 36180.789189999996, 40675.99635,
  28569.7197, 9253.896111, 36797.93332, 49357.19017, 15389.924680000002,
  20509.64777, 10808.47561, 9786.534714, 18678.31435, 25768.25759,
  28821.0637, 33859.74835, 37506.419069999996, 8458.276384, 33203.2612)

val lifeExpectancyEurope = Vector(76.423, 79.829, 79.441, 74.852, 73.005, 75.748, 76.486,
  78.332, 79.313, 80.657, 79.406, 79.483, 73.33800000000001, 81.757, 78.885, 80.546,
  74.543, 79.762, 80.196, 75.563, 78.098, 72.476, 74.002, 74.663, 77.926,
  80.941, 80.884, 81.70100000000001, 71.777, 79.425)

val labelEurope = Vector(
  "Albania",
  "Austria",
  "Belgium",
  "Bosnia and Herzegovina",
  "Bulgaria",
  "Croatia",
  "Czech Republic",
  "Denmark",
  "Finland",
  "France",
  "Germany",
  "Greece",
  "Hungary",
  "Iceland",
  "Ireland",
  "Italy",
  "Montenegro",
  "Netherlands",
  "Norway",
  "Poland",
  "Portugal",
  "Romania",
  "Serbia",
  "Slovak Republic",
  "Slovenia",
  "Spain",
  "Sweden",
  "Switzerland",
  "Turkey",
  "United Kingdom"
)

// Options common to both traces
val commonOptions = ScatterOptions().mode(ScatterMode.Marker).marker(MarkerOptions().size(12).lineWidth(1))

// Options common to both axes
val commonAxisOptions = AxisOptions().tickLength(5).gridWidth(2)

val xAxisOptions = commonAxisOptions.title("GDP per capita (dollars)").noZeroLine
val yAxisOptions = commonAxisOptions.title("Life expectancy (years)")

// The plot itself
val p = Plot().withScatter(gdpAmerica, lifeExpectancyAmerica,commonOptions.name("Americas").text(labelAmerica)).withScatter(gdpEurope, lifeExpectancyEurope,commonOptions.name("Europe").text(labelEurope)).xAxisOptions(xAxisOptions).yAxisOptions(yAxisOptions)
//val p = Plot().withScatter(gdpAmerica, lifeExpectancyAmerica,commonOptions.name("Americas").text(labelAmerica)).withScatter(gdpEurope, lifeExpectancyEurope,commonOptions.name("Europe").text(labelEurope))
val writer = FigureWriter()
val figure = Figure().plot(p).title("Life Expectancy v. Per Capita GDP, 2007")

draw(figure, p, "life-expectancy-per-GDP-2007",writer.FileOptions(overwrite=true))


Name: Compile Error
Message: <console>:25: error: not found: value ScatterOptions
       val commonOptions = ScatterOptions().mode(ScatterMode.Marker).marker(MarkerOptions().size(12).lineWidth(1))
                           ^
StackTrace: 